In [1]:
## Misc functions for data editing
# For HEA dataset
import os
import numpy as np
import pandas as pd
import pymatgen as mg
from pymatgen.core.composition import Composition
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Tick parameters
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['ytick.major.width'] = 1
plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['ytick.minor.width'] = 1

plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['legend.fontsize'] = 15


datafile_path = './'
df = pd.read_csv(datafile_path+'MPEA_dataset.csv')

def get_composition_df(formula):
    comp = Composition(formula)
    weights = [round(comp.get_atomic_fraction(ele), 3) for ele in comp.elements]
    elements = [str(x) for x in comp.elements]
    df = pd.DataFrame([weights], columns=elements)
    return df

comp_df = pd.concat([get_composition_df(x) for x in df['FORMULA']])
comp_df = comp_df.reindex(sorted(comp_df.columns), axis=1).fillna(0)

xls = pd.ExcelFile(datafile_path+'metal_property.xlsx')
df_descriptors = pd.read_excel(xls, 'Sheet1')
elemental_properties_dict = {}
elements_list = df_descriptors[df_descriptors.columns[0]]
row_number = 0
for element in elements_list:
    elemental_properties_dict[element] = [df_descriptors[i][row_number] for i in df_descriptors.columns[1:]]
    row_number += 1

comp_df[df_descriptors.columns[1:]] = 0.0
desc_num = 0
for descriptor in df_descriptors.columns[1:]:
    for element_name in elements_list:
        comp_df[descriptor] += comp_df[element_name]*elemental_properties_dict[element_name][desc_num]
    desc_num += 1

comp_df.index = df.index
comp_df['Processing'] = df['PROPERTY: Processing method']
comp_df['Target'] = df['PROPERTY: HV']

unique_alloys = comp_df.drop_duplicates(subset=df_descriptors.columns[1:])
unique_alloys.to_csv(datafile_path+'curated_MPEA_all629.csv',index=False)

# Shortlisted points having a hardness value
unique_alloys['Target'].replace('', np.nan)
unique_alloys_initial_training = unique_alloys.dropna(subset=['Target'])
unique_alloys_topperformers_training = unique_alloys_initial_training[unique_alloys_initial_training['Target'] > 850]

custom_order = ['CAST', 'ANNEAL', 'POWDER', 'WROUGHT', 'OTHER']

In [2]:
# Get the value counts of the 'Processing' column for all alloys
category_counts_all = unique_alloys['Processing'].value_counts()
category_counts_all = category_counts_all.reindex(custom_order) 

# Plot the bar chart
fig, ax = plt.subplots()
bars = ax.bar(category_counts_all.index, category_counts_all.values, color='b')

# Add count labels on top of bars
for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height}',             # text label
                xy=(bar.get_x() + bar.get_width() / 2, height),  # position at bar center top
                xytext=(0, 0),           # offset in points
                textcoords="offset points",
                ha='center', va='bottom', fontsize=12)

ax.set_xticklabels(category_counts_all.index, rotation=45, ha='right')
plt.xlabel('')
plt.ylabel('Count')

plt.tight_layout()
fig.patch.set_facecolor('white')
plt.savefig('./plots/full_dataset_processing.pdf', bbox_inches='tight')

# Get the value counts of the 'Processing' column for alloys with hardness values measured
category_counts_HV = unique_alloys_initial_training['Processing'].value_counts()
category_counts_HV = category_counts_HV.reindex(custom_order)

# Plot the bar chart
fig, ax = plt.subplots()
bars = ax.bar(category_counts_HV.index, category_counts_HV.values, color='b')

# Add count labels on top of bars
for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height}',             # text label
                xy=(bar.get_x() + bar.get_width() / 2, height),  # position at bar center top
                xytext=(0, 0),           # offset in points
                textcoords="offset points",
                ha='center', va='bottom', fontsize=12)
    
ax.set_xticklabels(category_counts_HV.index, rotation=45, ha='right')
plt.xlabel('')
plt.ylabel('Count')

plt.tight_layout()
fig.patch.set_facecolor('white')
plt.savefig('./plots/HV_processing.pdf', bbox_inches='tight')

/var/folders/02/mt5dp3vs5tgchb4s6sfwv5bc0000gp/T/ipykernel_28524/1408567416.py:18: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(category_counts_all.index, rotation=45, ha='right')
/var/folders/02/mt5dp3vs5tgchb4s6sfwv5bc0000gp/T/ipykernel_28524/1408567416.py:43: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(category_counts_HV.index, rotation=45, ha='right')


In [3]:
# Get the value counts of the 'Processing' column for alloys with high hardness values 
category_counts = unique_alloys_topperformers_training['Processing'].value_counts(sort=False)
category_counts['CAST'] = 8
category_counts['ANNEAL'] = 0
category_counts['POWDER'] = 1
category_counts['OTHER'] = 0
category_counts['WROUGHT'] = 0

category_counts = category_counts.reindex(custom_order)

# Plot the bar chart
fig, ax = plt.subplots()
bars = ax.bar(category_counts.index, category_counts.values, color='b')
# Add count labels on top of bars
for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height}',             # text label
                xy=(bar.get_x() + bar.get_width() / 2, height),  # position at bar center top
                xytext=(0, 0),           # offset in points
                textcoords="offset points",
                ha='center', va='bottom', fontsize=12)
    
ax.set_xticklabels(category_counts.index, rotation=45, ha='right')
plt.xlabel('')
plt.ylabel('Count')

plt.tight_layout()
fig.patch.set_facecolor('white')
plt.savefig('./plots/HV_topperformers_processing.pdf', bbox_inches='tight')

/var/folders/02/mt5dp3vs5tgchb4s6sfwv5bc0000gp/T/ipykernel_28524/3040165776.py:23: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(category_counts.index, rotation=45, ha='right')
